In [1]:
import sqlalchemy
sqlalchemy.__version__

'1.3.0'

In [2]:
from sqlalchemy import (create_engine, 
                        Column, 
                        Integer, String, Float, PickleType, ForeignKey)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

In [3]:
import numpy as np

In [4]:
from shapely.geometry import Point, Polygon

In [5]:
Base = declarative_base()

In [6]:
class Survey(Base):
    __tablename__ = 'surveys'
    
    id = Column(Integer, primary_key=True)
    name = Column('name', String(50), unique=True)
    
    # relationships
    area = relationship("Area", uselist=False, back_populates='survey')
    assemblage = relationship("Assemblage", uselist=False, back_populates='survey')
    coverage = relationship("Coverage", uselist=False, back_populates='survey')
    team = relationship("Team", uselist=False, back_populates='survey')

In [7]:
class Area(Base):
    __tablename__ = 'areas'
    
    id = Column(Integer, primary_key = True)
    name = Column('name', String(50), unique=True)
    vis = Column('vis', PickleType)
    shape = Column('shape', PickleType)
    
    # relationships
    survey_name = Column('survey_name', String(50), ForeignKey('surveys.id'))
    survey = relationship("Survey", back_populates='area')
    
    assemblages = relationship("Assemblage", back_populates='area')
#     layers = relationship("Layer", back_populates='area')
    coverage = relationship("Coverage", back_populates='area')

In [8]:
class Assemblage(Base):
    __tablename__ = 'assemblages'
    
    id = Column(Integer, primary_key = True)
    name = Column('name', String(50), unique=True)
    
    # relationships
    survey_name = Column('survey_name', String(50), ForeignKey('surveys.name'))
    survey = relationship("Survey", back_populates='assemblage')
    
    area_name = Column('area_name', String(50), ForeignKey('areas.name'))
    area = relationship("Area", back_populates='assemblages')
    
    layers = relationship("Layer", back_populates='assemblage')
    
    features = relationship("Feature", back_populates='assemblage')

In [9]:
class Layer(Base):
    __tablename__ = 'layers'
    
    id = Column(Integer, primary_key=True)
    name = Column('name', String(50), unique=True)
    feature_type = Column('feature_type', String(25))
    
    # relationships
#     area_name = Column('area_name', String(50), ForeignKey('areas.name'))
#     area = relationship("Area", back_populates='layers')
    
    assemblage_name = Column('assemblage_name', String(50), ForeignKey('assemblages.name'))
    assemblage = relationship("Assemblage", back_populates='layers')
    
    features = relationship("Feature", back_populates='layer')

In [10]:
class Feature(Base):
    __tablename__ = 'features'
    
    id = Column(Integer, primary_key=True)
    name = Column('name', String(50), unique=True)
    shape = Column('shape', PickleType)
    time_penalty = Column('time_penalty', PickleType)
    ideal_obs_rate = Column('ideal_obs_rate', PickleType)
    
    # relationships
    assemblage_name = Column('assemblage_name', String(50), ForeignKey('assemblages.name'))
    assemblage = relationship('Assemblage', back_populates='features')
    
    layer_name = Column('layer_name', String(50), ForeignKey('layers.name'))
    layer = relationship('Layer', back_populates='features')

In [11]:
class Coverage(Base):
    __tablename__ = 'coverages'

    id = Column(Integer, primary_key=True)
    name = Column('name', String(50), unique=True)    
    survey_unit_type = Column('survey_unit_type', String(50))
    orientation = Column('orientation', Float)
    spacing = Column('spacing', Float)
    sweep_width = Column('sweep_width', Float, default=None)
    radius = Column('radius', Float, default=None)
    
    # relationships
    survey_name = Column('survey_name', String(50), ForeignKey('surveys.name'))
    survey = relationship("Survey", back_populates='coverage')
    
    area_name = Column('area_name', String(50), ForeignKey('areas.name'))
    area = relationship("Area", back_populates='coverage')

    surveyunit = relationship('SurveyUnit', back_populates='coverage')

In [12]:
class SurveyUnit(Base):
    __tablename__ = 'surveyunits'
    
    id = Column(Integer, primary_key=True)
    name = Column('name', String(50), unique=True)
    min_time_per_unit = Column('min_time_per_unit', Float)
    shape = Column('shape', PickleType)
    
    # relationships
    coverage_name = Column('coverage_name', String(50), ForeignKey('coverages.name'))
    coverage = relationship('Coverage', back_populates='surveyunit')

In [13]:
class Team(Base):
    __tablename__ = 'teams'
    
    id = Column(Integer, primary_key=True)
    name = Column('name', String(50), unique=True)
    
    # relationships
    survey_name = Column('survey_name', String(50), ForeignKey('surveys.name'))
    survey = relationship("Survey", back_populates='team')

    surveyors = relationship("Surveyor", back_populates='team')

In [14]:
class Surveyor(Base):
    __tablename__ = 'surveyors'
    
    id = Column(Integer, primary_key=True)
    name = Column('name', String(50), unique=True)    
    surveyor_type = Column('surveyor_type', String(50))
    skill = Column('skill', PickleType)
    speed_penalty = Column('speed_penalty', PickleType)

    # relationships
    team_name = Column('team_name', String(50), ForeignKey('teams.name'))
    team = relationship("Team", back_populates='surveyors')


In [15]:
engine = create_engine('sqlite:///sqlalchemy_test.db')

# THIS IS VERY IMPORTANT FOR SQLALCHEMY
Base.metadata.create_all(engine)

In [19]:
survey_test = Survey(name='survey_test')
area_test = Area(name='area_test', 
                 vis=np.ndarray(50), 
                 shape=np.random.rand(50), survey_name='survey_test')

In [20]:
Session = sessionmaker(bind=engine)
session = Session()

In [21]:
session.add(survey_test)
session.add(area_test)
session.commit()

In [22]:
session.query(Survey).filter_by(name='survey_test').first()

In [1]:
import numpy as np

In [4]:
type(np.array([1.0]))

numpy.ndarray

In [15]:
import pandas as pd

class Child():
    def __init__(self, name, text):
        self.name = name
        self.text = text
    
    def to_dict(self):
        return {
            'name': self.name,
            'text': self.text
        }

class Parent():
    def __init__(self, child_list):
        self.child_list = child_list
        
        self.df = pd.DataFrame([k.to_dict() for k in self.child_list])

In [16]:
kids = [Child(f'child_{i}', f'text_{i}') for i in range(5)]

In [17]:
kids

In [18]:
p = Parent(kids)

In [19]:
p.df

,name,text
0,child_0,text_0
1,child_1,text_1
2,child_2,text_2
3,child_3,text_3
4,child_4,text_4


In [23]:
import geopandas as gpd

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

world.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,28400000.0,Asia,Afghanistan,AFG,22270.0,"POLYGON ((61.21081709172574 35.65007233330923,..."
1,12799293.0,Africa,Angola,AGO,110300.0,(POLYGON ((16.32652835456705 -5.87747039146621...
2,3639453.0,Europe,Albania,ALB,21810.0,"POLYGON ((20.59024743010491 41.85540416113361,..."
3,4798491.0,Asia,United Arab Emirates,ARE,184300.0,"POLYGON ((51.57951867046327 24.24549713795111,..."
4,40913584.0,South America,Argentina,ARG,573900.0,(POLYGON ((-65.50000000000003 -55.199999999999...


In [ ]:
shape = 